In [7]:
import os

from dotenv import load_dotenv
from langchain import hub
from langchain_community.document_loaders import Docx2txtLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pinecone import Pinecone


## 환경변수 읽어오기
load_dotenv()
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
LANGCHAIN_API_KEY = os.getenv('LANGCHAIN_API_KEY')


In [6]:
## 문서 읽고 분할 #####################################################
from langchain_community.document_loaders import PyPDFLoader
## 문서 파일 목록
pdf_paths = ['스페셜리포트_2024-Vol.3-AI란-무엇인가.pdf', '스페셜리포트_2025년10대미래기술.pdf']

## 문서 로드(읽어오기)
documents = []

for path in pdf_paths:
    loader = PyPDFLoader(path)
    documents.extend(loader.load())

documents

[Document(metadata={'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 19.5 (Windows)', 'creationdate': '2024-08-12T11:26:00+09:00', 'moddate': '2024-08-12T11:26:32+09:00', 'trapped': '/False', 'source': '스페셜리포트_2024-Vol.3-AI란-무엇인가.pdf', 'total_pages': 39, 'page': 0, 'page_label': '1'}, page_content='Special Report\nWhat is AI?\nAI란 무엇인가?\n2024 volume.3'),
 Document(metadata={'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 19.5 (Windows)', 'creationdate': '2024-08-12T11:26:00+09:00', 'moddate': '2024-08-12T11:26:32+09:00', 'trapped': '/False', 'source': '스페셜리포트_2024-Vol.3-AI란-무엇인가.pdf', 'total_pages': 39, 'page': 1, 'page_label': '2'}, page_content='AI는 이제 알게 모르게 거의 모든 사람들이 사용하는 기술로 자리잡고 있습니다. 스마트폰의 사진 보\n정 앱에서부터, 온라인 쇼핑몰의 상품 추천, 인터넷 검색 엔진에 이르기까지 AI의 역할은 끊임없이 \n확장되고 있습니다. 하지만 과연 우리는 이런 AI에 대해 얼마나 알고 있을까요?\n모두가 AI가 무엇인지 알고 있다고 생각하지만 각자 정의하는 AI에는 큰 차이가 있습니다. AI의 과\n거부터 현재까지를 짚어보면서 AI가 진정으로 무엇인지 이해해 보는 시간을 가져 보겠습니다.'),
 Document(metadata={'producer': 'Adobe 

In [8]:
## 청크 분할 설정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

## 문서 분할
document_list = text_splitter.split_documents(documents)

len(document_list)

72

In [9]:

## 임베딩 -> 벡터 스토어(데이터베이스)에 저장
## 임베딩 모델 지정
embedding = OpenAIEmbeddings(model='text-embedding-3-large')
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = 'project'

# 파인콘: 저장
database =  PineconeVectorStore.from_documents(
    documents=document_list,
    embedding=embedding,
    index_name=index_name,
)

## 저장된 인덱스 가져오기
## [방법 1]
# database = PineconeVectorStore(
#     index=pc.Index(index_name),
#     embedding=embedding,
# )

## [방법 2]
# database = PineconeVectorStore.from_existing_index(
#     index_name=index_name,
#     embedding=embedding,
# )

